# MPS canonical forms 

In [17]:

import numpy as np
from numpy import linalg as LA
from tensornetwork import ncon


In [18]:
lis1 = [5]* 10
print(lis1)

[5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


In [19]:
# MPS Builder form - OBC for now 
def buildMPS(LL: int, chi: int=10, d: int=2):

    print(f"Building MPS with length {LL}, chi = {chi} and physical d={d}")

    myChi = [chi] * (LL-1)
    mySV = [1] * (LL-1)

    # Build the first element of the MPS A_1 (actually 0th elem of the list)
    myMPS = [ np.random.rand(d,chi) + 1j*np.random.rand(d,chi) ]

    #Build all the others, A_2 to A_(N-1)
    for ii in range(2,LL):  # 2 to N-1, so actually building N-2 elements
         myMPS.append(np.random.rand(chi,d,chi) + 1j*np.random.rand(chi,d,chi) )
 
    # Build the Nth element (actually element N-1 of the list)
    myMPS.append( np.random.rand(chi,d) + 1j*np.random.rand(chi,d))

    return myMPS, myChi, mySV


In [20]:
class randMPS:                   
 
    def __init__(self, LL: int, chi: int=10, d: int=2):

        print(f"Building MPS with length {LL}, chi = {chi} and physical d={d}")

        mChi = [chi] * (LL-1)
        mSV = [1] * (LL-1)

        # Build the first element of the MPS A_1 (actually 0th elem of the list)
        mMPS = [ np.random.rand(d,chi) + 1j*np.random.rand(d,chi) ]

        #Build all the others, A_2 to A_(N-1)
        for _ in range(2,LL):  # 2 to N-1, so actually building N-2 elements
            mMPS.append(np.random.rand(chi,d,chi) + 1j*np.random.rand(chi,d,chi) )
    
        # Build the Nth element (actually element N-1 of the list)
        mMPS.append( np.random.rand(chi,d) + 1j*np.random.rand(chi,d))

    
        self.MPS = mMPS  
        self.d = d
        self.LL = LL     
        self.chi = mChi
        self.SV = mSV  # Probably unnecessary
        

In [21]:
import logging

def bringLeftCan(MPS: object):

    """ 
    Performs LEFT sweep of MPS 
    """
    
    Slist = MPS.SV
  

    chilist = MPS.chi

    Vdaglist = []  # Unnecessary?
    Alist = []

    # First site:

    U, S, Vdag = LA.svd(MPS[0],full_matrices=0)

    logging.info("First SVD:")
    logging.info( f"{np.shape(MPS[0])} = {np.shape(U)} . {np.shape(S)} . {np.shape(Vdag)}")
    logging.info(f"SV = {S}")
    logging.info( f"{aMPS[0]} = {U} . {S} . {Vdag}")
    logging.info(f"r[0] (nonzero SVs) = {np.size(S)}")

    chilist[0] = np.size(S)

    Slist[0] = np.diag(S)  # We build a matrix from the SVDs

    Vdaglist.append(Vdag)

    Alist.append(U)

    """ 
    Going to the next one, we need to make some products and some reshaping.
    We work with 
    """

    for jj in range(1,NN-1):
        print(f"Building leftcanon form for M[{jj}]. First build Mtilde = SV.Vdag.M[{jj}]")
        SVdag = Slist[-1] @ Vdaglist[-1]
        
        print(f"SV = {np.shape(Slist[-1])} , Vdag = {np.shape(Vdaglist[-1])}")
        print(f"SVdag = {np.shape(SVdag)} , M{jj} = {np.shape(aMPS[jj])}")

        Mtilde = ncon([SVdag, aMPS[jj]], [[-1,1],[1,-2,-3]])  # TODO: CHECK THIS PROD

        print(f"Mtilde = {np.shape(Mtilde)}")
        Mtr = np.reshape(Mtilde, (chilist[-1]*d, chi))

        U, S, Vdag = LA.svd(Mtr,full_matrices=0)  #TODO: work with compressed matrices?
        
        print( f"SVD: {np.shape(Mtr)} = {np.shape(U)} . {np.shape(S)} . {np.shape(Vdag)}")

        print(f"r[{jj}] (nonzero SVs) = {np.size(S)}")

        chilist.append(np.size(S))

        # Reshape U

        U = np.reshape(U,(chilist[-2],d,chilist[-1]))

        Ulist.append(U)
        Slist.append(np.diag(S))  # We build a matrix from the SVDs
        Vdaglist.append(Vdag)

        Alist.append(U)


    # Now the last site ("L-1")
    print(f"Last site is just a vector, ie. tensor has shape {np.shape(aMPS[NN-1])}")

    SVdag = Slist[-1] @ Vdaglist[-1]
      
    print(f"SV = {np.shape(Slist[-1])} , Vdag = {np.shape(Vdaglist[-1])}")
    print(f"SVdag = {np.shape(SVdag)} , M{NN-1} = {np.shape(aMPS[NN-1])}")

    Mtilde = ncon([SVdag, aMPS[NN-1]], [[-1,1],[1,-2]])  # TODO: CHECK THIS PROD

    print(f"Mtilde = {np.shape(Mtilde)}")

    # We should still reshape here!
    Mtr = np.reshape(Mtilde, (chilist[-1]*d, 1))

    U, S, Vdag = LA.svd(Mtr,full_matrices=0)  #TODO: work with compressed matrices?
      
    print( f"SVD: {np.shape(Mtr)} = {np.shape(U)} . {np.shape(S)} . {np.shape(Vdag)}")

    print(f"r[{jj}] (nonzero SVs) = {np.size(S)}")

    # We don't need the last chi, it's just 1 
    #chilist.append(np.size(S))

    # I guess we can reshape the last simply as

    print(f"From {np.shape(U)} ")
    #U = np.reshape(U,(chilist[-2],d,chilist[-1]))
    U = np.reshape(U,(chilist[-2],d))
    print(f"to {np.shape(U)} ")

    Alist.append(U)

    print(f"Built a list of {len(Alist)} left-canon tensors (cfr. L = {NN} )")
    print(f"And a list of {len(chilist)} bond dimensions")

    # The last factor should give the normalization

    tail = S @ Vdag 

    print(tail, tail*tail)

    # So we can just chuck it away I guess..


In [12]:

mMPS, mChi, mSV = buildMPS(20)

Building MPS with length 20, chi = 10 and physical d=2
